## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [5]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [6]:
load_dotenv(override=True)

True

In [7]:
# Let's just check emails are working for you

def send_test_email():
    # First, check if API key is properly loaded
    api_key = os.environ.get('SENDGRID_API_KEY')
    
    if not api_key:
        print("❌ Error: SENDGRID_API_KEY is not set in environment variables")
        print("\nPlease check the following:")
        print("1. Verify that a .env file exists in the project root directory")
        print("2. Verify that the .env file contains the following line:")
        print("   SENDGRID_API_KEY=your_api_key_here")
        print("3. Verify that load_dotenv() has been executed (Cell 3)")
        print("4. Verify that the API key has been created in the SendGrid dashboard")
        return
    
    # Check if API key looks valid (should start with SG.)
    if not api_key.startswith('SG.'):
        print(f"⚠️  Warning: API key doesn't start with 'SG.' (first 10 chars: {api_key[:10]}...)")
        print("   This might indicate an invalid API key format.")
    
    # Display only the first few characters of the API key (for security)
    print(f"✓ API key loaded successfully (first 10 chars: {api_key[:10]}...)")
    print(f"✓ API key length: {len(api_key)} characters")
    
    try:
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        from_email = Email("vr.work.ams@gmail.com")  # Change to your verified sender
        to_email = To("vr.work.ams@gmail.com")  # Change to your recipient
        content = Content("text/plain", "This is an important test email")
        mail = Mail(from_email, to_email, "Test email", content).get()
        response = sg.client.mail.send.post(request_body=mail)
        print(f"✓ Email sent successfully! Status code: {response.status_code}")
        if response.status_code == 202:
            print("✓ Email was sent successfully (202 = Accepted)")
    except Exception as e:
        error_type = type(e).__name__
        error_msg = str(e)
        print(f"❌ Error occurred: {error_type}")
        print(f"   Details: {error_msg}")
        
        # Specific handling for 401 Unauthorized
        if "401" in error_msg or "Unauthorized" in error_type:
            print("\n🔍 401 Unauthorized Error - Common causes:")
            print("1. ❌ Invalid API Key:")
            print("   - Go to SendGrid Dashboard > Settings > API Keys")
            print("   - Verify your API key is correct and active")
            print("   - Make sure you copied the FULL key (it should start with 'SG.' and be ~70 chars)")
            print("   - Try creating a NEW API key with 'Full Access' permissions")
            print("\n2. ❌ API Key Permissions:")
            print("   - The API key must have 'Mail Send' permissions")
            print("   - When creating the key, select 'Full Access' or ensure 'Mail Send' is enabled")
            print("\n3. ❌ Sender Email Not Verified:")
            print("   - Go to SendGrid Dashboard > Settings > Sender Authentication")
            print("   - Click 'Verify a Single Sender'")
            print("   - Verify the email: vr.work.ams@gmail.com")
            print("   - Check your email inbox for the verification link")
            print("\n4. ❌ Account Issues:")
            print("   - Check if your SendGrid account is active")
            print("   - Free tier accounts may have restrictions")
            print("   - Verify your account email is confirmed")
        else:
            print("\nTroubleshooting:")
            print("1. Verify that your SendGrid API key is valid")
            print("2. Verify that the sender email address (vr.work.ams@gmail.com) is authenticated in SendGrid")
            print("3. Check the API key permissions in the SendGrid dashboard")
            print("4. Check your internet connection and firewall settings")

send_test_email()

✓ API key loaded successfully (first 10 chars: SG.aUkygqb...)
✓ API key length: 69 characters
✓ Email sent successfully! Status code: 202
✓ Email was sent successfully (202 = Accepted)


In [8]:
# Diagnostic: Check SendGrid API Key Configuration
# Run this cell to diagnose API key issues before sending emails

import os
import sendgrid

def diagnose_sendgrid_config():
    """Diagnose SendGrid API key and configuration issues"""
    print("=" * 60)
    print("SendGrid Configuration Diagnostic")
    print("=" * 60)
    
    # Check if API key exists
    api_key = os.environ.get('SENDGRID_API_KEY')
    if not api_key:
        print("❌ SENDGRID_API_KEY is NOT set in environment variables")
        print("\n📝 To fix this:")
        print("   1. Open your .env file in the project root")
        print("   2. Add this line: SENDGRID_API_KEY=your_actual_key_here")
        print("   3. Make sure load_dotenv() has been executed")
        return False
    
    print(f"✓ SENDGRID_API_KEY is set")
    print(f"  - Length: {len(api_key)} characters")
    print(f"  - First 10 chars: {api_key[:10]}...")
    print(f"  - Last 10 chars: ...{api_key[-10:]}")
    
    # Check API key format
    if api_key.startswith('SG.'):
        print("✓ API key format looks correct (starts with 'SG.')")
    else:
        print("⚠️  WARNING: API key doesn't start with 'SG.'")
        print("   This might indicate an incorrect key format.")
        print("   Valid SendGrid API keys typically start with 'SG.'")
    
    # Check key length (SendGrid keys are usually ~70 characters)
    if len(api_key) < 50:
        print("⚠️  WARNING: API key seems too short (expected ~70 characters)")
    elif len(api_key) > 100:
        print("⚠️  WARNING: API key seems too long (expected ~70 characters)")
    else:
        print("✓ API key length looks reasonable")
    
    # Check for common issues
    if ' ' in api_key or '\n' in api_key:
        print("⚠️  WARNING: API key contains spaces or newlines")
        print("   Make sure there are no extra spaces in your .env file")
        print("   Format should be: SENDGRID_API_KEY=SG.xxxxx (no spaces around =)")
    
    # Try to create a client (this doesn't make an API call)
    try:
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        print("✓ SendGrid client created successfully")
    except Exception as e:
        print(f"❌ Failed to create SendGrid client: {e}")
        return False
    
    print("\n" + "=" * 60)
    print("Next Steps:")
    print("=" * 60)
    print("1. If all checks pass, try running the send_test_email() function")
    print("2. If you get a 401 error, check:")
    print("   - SendGrid Dashboard > Settings > API Keys")
    print("   - Make sure your API key has 'Mail Send' permissions")
    print("   - Verify your sender email in Settings > Sender Authentication")
    print("3. Create a new API key with 'Full Access' if issues persist")
    
    return True

# Run the diagnostic
diagnose_sendgrid_config()


SendGrid Configuration Diagnostic
✓ SENDGRID_API_KEY is set
  - Length: 69 characters
  - First 10 chars: SG.aUkygqb...
  - Last 10 chars: ...3mhHDY3GaY
✓ API key format looks correct (starts with 'SG.')
✓ API key length looks reasonable
✓ SendGrid client created successfully

Next Steps:
1. If all checks pass, try running the send_test_email() function
2. If you get a 401 error, check:
   - SendGrid Dashboard > Settings > API Keys
   - Make sure your API key has 'Mail Send' permissions
   - Verify your sender email in Settings > Sender Authentication
3. Create a new API key with 'Full Access' if issues persist


True

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [9]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [10]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [11]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Elevate Your Compliance Efforts with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit preparation.

In today's regulatory environment, ensuring compliance is critical for building trust with clients and safeguarding your organization. ComplAI leverages AI technology to simplify the complexities of SOC 2 requirements, providing real-time insights and automating documentation processes. 

Here are a few key benefits of using ComplAI:
- **Automated Workflows**: Reduce the manual effort involved in compliance tasks, allowing your team to focus on what they do best.
- **Real-Time Monitoring**: Stay ahead of compliance requirements with continuous monitoring and alerts tailored to your specific needs.
- **Comprehensive Reporting**: Generate detailed reports effortlessly, making audits not just manageable but stra

In [12]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI, where we specialize in simplifying the SOC 2 compliance process for organizations like yours.

Achieving and maintaining SOC 2 compliance can often feel overwhelming, especially when dealing with the ever-evolving regulatory landscape. ComplAI offers an AI-powered SaaS solution that not only streamlines the compliance process but also prepares your team for successful audits, making it more efficient and less time-consuming.

Here are a few key benefits of our platform:

- **Automated Documentation**: Generate the necessary documentation effortlessly, ensuring that nothing is overlooked.
- **Real-time Monitoring**: Stay ahead of compliance requirements with continuous updates and alerts, tailored specifically for your organization.
- **Audit Preparation**: Our guided approach helps you to be thoroughly prepared for any

In [13]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [14]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Does Your SOC2 Compliance Need a Dating Coach? 💔➡️💖

Hey [Recipient's Name],

Ever tried to help your compliance team find love? I mean, there’s navigating the tumultuous waters of audits, cringing at “Did we really forget that?” moments... it’s a real “hearts and minds” game!

Here at ComplAI, we don’t have a magic wand, but we do have a smart SaaS tool that puts romance back in your compliance game! Think of us as the matchmaker that ensures your SOC2 audit is a match made in heaven—no awkward first dates with the auditors!

Why choose ComplAI?

1. **AI-Powered Insights:** No more guessing games. Our AI dives deep so you don’t have to dig through mountains of paperwork.
  
2. **Compliance Made Easy:** We take the headache out of audits. The only thing you need to worry about is which pizza to order for the team!

3. **Love at First Sight:** Our user-friendly interface is so easy to navigate, you’ll feel like you’ve known it forever.

Ready to stop swiping l

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [15]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [16]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [17]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("vr.work.ams@gmail.com")  # Change to your verified sender
    to_email = To("vr.work.ams@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [18]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed5d1c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### And you can also convert an Agent into a tool

In [19]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x111a34d60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [20]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1118ffce0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1119574c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='Write 

## And now it's time for our Sales Manager - our planning agent

In [21]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [22]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [23]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("vr.work.ams@gmail.com")  # Change to your verified sender
    to_email = To("vr.work.ams@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [24]:
tools = [subject_tool, html_tool, send_html_email]

In [25]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x111a34ea0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1101163e0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 Function

In [26]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [27]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1118ffce0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1119574c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', description='Write a 

In [28]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

## Bonus: Email Reply Webhook - Automated SDR Responses

This section implements the HARD CHALLENGE from the exercise: setting up a webhook so SendGrid notifies us when someone replies to an email, and then having the SDR agent automatically respond to keep the conversation going.

### ⚠️ Important: Two Types of Webhooks

**1. Event Webhook** (Recommended for testing without a domain):
- Tracks email events: delivered, opened, clicked, bounced, etc.
- **Does NOT receive actual email replies** - only metadata
- Works with ngrok (no domain needed)
- Good for tracking engagement

**2. Inbound Parse** (For receiving actual email replies):
- Receives the full email content when someone replies
- **Requires a domain** (can't use Gmail directly)
- Needs DNS configuration
- Required for automated reply functionality

### 🚀 Quick Start: Event Webhook Setup (No Domain Required!)

Since you're using Gmail and don't have a domain, here's how to set up an **Event Webhook** for testing:


In [29]:
# Quick Fix: Install Flask using uv (recommended for this project)
# Run this cell if Flask import fails above

import sys
import subprocess
import os

print(f"Python executable: {sys.executable}")
print(f"Project directory: {os.getcwd()}")

# Try using uv first (since this project uses uv)
print("\n🔧 Attempting to install Flask using uv...")
try:
    # Use uv pip install (works better with uv projects)
    result = subprocess.run(
        ["uv", "pip", "install", "flask"],
        cwd=os.getcwd(),
        capture_output=True,
        text=True,
        timeout=60
    )
    
    if result.returncode == 0:
        print("✓ Flask installed successfully using uv!")
        print("\n⚠️  IMPORTANT: Restart the kernel now:")
        print("   - Go to: Kernel → Restart Kernel")
        print("   - Then re-run the import cell (Cell 46)")
    else:
        print(f"⚠️  uv installation had issues:")
        print(result.stderr)
        raise Exception("uv install failed")
        
except FileNotFoundError:
    print("⚠️  uv not found in PATH. Trying pip instead...")
    try:
        # Fallback to pip
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "flask"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=60
        )
        print("✓ Flask installed successfully using pip!")
        print("\n⚠️  IMPORTANT: Restart the kernel now:")
        print("   - Go to: Kernel → Restart Kernel")
        print("   - Then re-run the import cell (Cell 46)")
    except Exception as e:
        print(f"❌ Both uv and pip installation failed")
        print(f"\n💡 Try installing manually in a terminal:")
        print(f"   cd {os.getcwd()}")
        print(f"   uv pip install flask")
        print(f"\n   Or if uv is not available:")
        print(f"   {sys.executable} -m pip install flask")
        print(f"\n   Then restart the kernel and re-run Cell 46")
except subprocess.TimeoutExpired:
    print("❌ Installation timed out. Try installing manually:")
    print(f"   cd {os.getcwd()}")
    print(f"   uv pip install flask")
except Exception as e:
    print(f"❌ Installation failed: {e}")
    print(f"\n💡 Try installing manually in a terminal:")
    print(f"   cd {os.getcwd()}")
    print(f"   uv pip install flask")
    print(f"\n   Then restart the kernel and re-run Cell 46")


Python executable: /Users/vijayr/Documents/Projects/agents/.venv/bin/python
Project directory: /Users/vijayr/Documents/Projects/agents/2_openai

🔧 Attempting to install Flask using uv...
✓ Flask installed successfully using uv!

⚠️  IMPORTANT: Restart the kernel now:
   - Go to: Kernel → Restart Kernel
   - Then re-run the import cell (Cell 46)


In [2]:
# Additional imports for webhook functionality

import sys
import subprocess

# Diagnostic: Check Python environment
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")

# Try to import Flask
try:
    from flask import Flask, request, jsonify
    import flask
    print(f"✓ Flask is installed and imported successfully! (version {flask.__version__})")
    Flask = Flask  # Make it available for other cells
except ImportError:
    print("⚠️  Flask not found in current Python environment")
    print(f"   Current Python: {sys.executable}")
    print("\n💡 Solutions:")
    print("   1. Run Cell 45 to install Flask using uv")
    print("   2. Or install manually in terminal:")
    print("      cd /Users/vijayr/Documents/Projects/agents")
    print("      uv pip install flask")
    print("   3. Then RESTART the notebook kernel (Kernel → Restart)")
    print("   4. Re-run this cell")
    Flask = None

import re
import json
from datetime import datetime
from email.utils import parseaddr

# For local development, you can use Flask
# For production, consider using a more robust framework or serverless function


Python executable: /Users/vijayr/Documents/Projects/agents/.venv/bin/python
Python version: 3.12.12
✓ Flask is installed and imported successfully! (version 3.1.2)


/var/folders/0d/vq9qypqj3_l7kg5skqtmmx8c0000gn/T/ipykernel_37854/2143932607.py:14: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.2. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(f"✓ Flask is installed and imported successfully! (version {flask.__version__})")


In [30]:
# Helper functions for processing email replies

def extract_email_address(email_string):
    """Extract clean email address from 'Name <email@domain.com>' format"""
    name, email = parseaddr(email_string)
    return email if email else email_string

def clean_reply_text(text):
    """Remove quoted text and email headers from reply"""
    if not text:
        return ""
    
    # Remove common email reply patterns
    lines = text.split('\n')
    cleaned_lines = []
    in_quoted_section = False
    
    for line in lines:
        # Detect start of quoted section
        if line.strip().startswith('>') or \
           line.strip().startswith('On ') and 'wrote:' in line or \
           line.strip().startswith('From:') or \
           line.strip().startswith('Sent:') or \
           '-----Original Message-----' in line:
            in_quoted_section = True
            continue
        
        # Stop if we hit a separator
        if '---' in line and len(line.strip()) < 10:
            break
            
        if not in_quoted_section:
            cleaned_lines.append(line)
    
    cleaned = '\n'.join(cleaned_lines).strip()
    # Remove excessive whitespace
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned)
    return cleaned

def extract_subject_from_headers(headers):
    """Extract subject from email headers"""
    if isinstance(headers, str):
        # Try to parse if it's a JSON string
        try:
            headers = json.loads(headers)
        except:
            pass
    
    if isinstance(headers, dict):
        return headers.get('subject', 'Re: Your inquiry')
    return 'Re: Your inquiry'


In [31]:
# Create SDR Response Agent - handles email replies intelligently

sdr_response_instructions = """
You are an intelligent Sales Development Representative (SDR) for ComplAI, 
a SaaS company providing SOC2 compliance tools powered by AI.

Your role is to respond to email replies from prospects in a natural, conversational way.

Guidelines:
1. **Be conversational and human-like** - Don't sound robotic
2. **Answer questions directly** - If they ask about pricing, features, or timeline, provide helpful information
3. **Read the context** - Understand what they're responding to from the original email
4. **Move the conversation forward** - Ask follow-up questions, suggest next steps, or offer to schedule a call
5. **Be concise** - Keep responses to 2-4 paragraphs, suitable for email
6. **Match their tone** - If they're casual, be casual. If formal, be professional
7. **Handle objections gracefully** - If they're not interested, be respectful and leave the door open

Important context about ComplAI:
- We help companies prepare for SOC2 audits
- Our tool is AI-powered and automates compliance workflows
- We reduce audit preparation time significantly
- We offer demos and free consultations

Always sign off professionally with your name and title.
"""

sdr_response_agent = Agent(
    name="SDR Response Agent",
    instructions=sdr_response_instructions,
    model="gpt-4o-mini"
)


In [32]:
# Function to send response email via SendGrid

@function_tool
def send_reply_email(to_email: str, subject: str, body: str, original_subject: str = None) -> Dict[str, str]:
    """
    Send a reply email to a prospect who responded to our sales email.
    
    Args:
        to_email: Recipient email address
        subject: Email subject line
        body: Email body (plain text)
        original_subject: Original email subject for threading
    """
    try:
        api_key = os.environ.get('SENDGRID_API_KEY')
        if not api_key:
            return {"status": "error", "message": "SENDGRID_API_KEY not set"}
        
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        
        # Use your verified sender email
        from_email = Email("vr.work.ams@gmail.com")  # Change to your verified sender
        to_email_obj = To(to_email)
        
        # Create email content
        content = Content("text/plain", body)
        
        # Create mail object
        mail = Mail(from_email, to_email_obj, subject, content)
        
        # Set reply-to for proper threading (optional, helps with email threading)
        if original_subject:
            mail.reply_to = Email("vr.work.ams@gmail.com")
        
        # Send email
        response = sg.client.mail.send.post(request_body=mail.get())
        
        return {
            "status": "success",
            "message": f"Reply sent to {to_email}",
            "status_code": response.status_code
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
# Process incoming email reply and generate response

async def process_email_reply(from_email: str, reply_text: str, original_subject: str = None):
    """
    Process an incoming email reply and generate an appropriate response using the SDR agent.
    
    Args:
        from_email: Email address of the person who replied
        reply_text: The text content of their reply (cleaned)
        original_subject: Original email subject for context
    """
    try:
        # Build context for the SDR agent
        context = f"""
        You received a reply from: {from_email}
        
        Original subject: {original_subject or 'Sales inquiry'}
        
        Their reply:
        {reply_text}
        
        Generate a natural, helpful response that:
        1. Acknowledges their message
        2. Answers any questions they asked
        3. Moves the conversation forward
        4. Is appropriate for email (2-4 paragraphs)
        
        Write your response as if you're the SDR responding directly to them.
        """
        
        # Use the SDR agent to generate response
        result = await Runner.run(sdr_response_agent, context)
        response_text = result.final_output
        
        # Generate appropriate subject line
        if original_subject:
            if not original_subject.startswith('Re:'):
                subject = f"Re: {original_subject}"
            else:
                subject = original_subject
        else:
            subject = "Re: Your inquiry about ComplAI"
        
        # Send the reply
        send_result = send_reply_email(
            to_email=from_email,
            subject=subject,
            body=response_text,
            original_subject=original_subject
        )
        
        print(f"✓ Processed reply from {from_email}")
        print(f"✓ Response sent: {send_result.get('status')}")
        
        return {
            "status": "success",
            "response_sent": True,
            "recipient": from_email
        }
        
    except Exception as e:
        print(f"❌ Error processing reply: {str(e)}")
        return {
            "status": "error",
            "message": str(e)
        }


In [ ]:
# Flask webhook server to receive SendGrid webhooks

if Flask is None:
    print("❌ Flask is not installed. Please install it first:")
    print("   pip install flask")
    print("   Or: uv pip install flask")
    print("\n⚠️  The webhook server code below will not work until Flask is installed.")
    app = None
else:
    app = Flask(__name__)

    @app.route('/webhook/email', methods=['POST'])
    def handle_inbound_email():
        """
        Handle incoming webhooks from SendGrid.
        
        This endpoint handles TWO types of webhooks:
        1. Event Webhooks - JSON array of events (delivered, opened, clicked, etc.)
        2. Inbound Parse - Form data with email content (requires domain)
        """
        try:
            # Check if it's an Event Webhook (JSON array)
            if request.is_json:
                events = request.json
                # SendGrid sends an array of events
                if isinstance(events, list):
                    print(f"\n📬 Received {len(events)} event(s) from SendGrid")
                    results = []
                    for event in events:
                        result = handle_sendgrid_event(event)
                        results.append(result)
                    return jsonify({"status": "processed", "events": results}), 200
                else:
                    # Single event
                    result = handle_sendgrid_event(events)
                    return jsonify({"status": "processed", "event": result}), 200
            
            # Otherwise, treat as Inbound Parse (form data with email content)
            # This requires a domain setup
            from_email_raw = request.form.get('from', '')
            subject = request.form.get('subject', 'Re: Your inquiry')
            text_body = request.form.get('text', '')
            html_body = request.form.get('html', '')
            headers = request.form.get('headers', '{}')
            
            # Extract clean email address
            from_email = extract_email_address(from_email_raw)
            
            # Clean the reply text (remove quoted content)
            cleaned_text = clean_reply_text(text_body)
            
            # Log the incoming email
            print(f"\n📧 Received email reply (Inbound Parse):")
            print(f"   From: {from_email}")
            print(f"   Subject: {subject}")
            print(f"   Cleaned text length: {len(cleaned_text)} chars")
            
            # Process asynchronously
            import asyncio
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            result = loop.run_until_complete(
                process_email_reply(from_email, cleaned_text, subject)
            )
            loop.close()
            
            return jsonify({
                "status": "received",
                "processed": True,
                "result": result
            }), 200
            
        except Exception as e:
            print(f"❌ Error handling webhook: {str(e)}")
            import traceback
            traceback.print_exc()
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500

    @app.route('/webhook/email', methods=['GET'])
    def webhook_info():
        """Info endpoint to verify webhook is accessible"""
        return jsonify({
            "status": "webhook_endpoint_active",
            "message": "SendGrid webhook endpoint is ready",
            "endpoints": {
                "POST /webhook/email": "Receives SendGrid Event Webhooks and Inbound Parse",
                "GET /webhook/email": "This info endpoint",
                "GET /health": "Health check"
            },
            "note": "For Event Webhooks, SendGrid sends JSON. For Inbound Parse, SendGrid sends form data."
        }), 200
        """
        Handle incoming email webhook from SendGrid Inbound Parse.
        
        SendGrid sends form data with email fields:
        - from: Sender email address
        - subject: Email subject
        - text: Plain text body
        - html: HTML body (optional)
        - headers: Email headers (JSON string)
        """
        try:
            # Get form data from SendGrid
            from_email_raw = request.form.get('from', '')
            subject = request.form.get('subject', 'Re: Your inquiry')
            text_body = request.form.get('text', '')
            html_body = request.form.get('html', '')
            headers = request.form.get('headers', '{}')
            
            # Extract clean email address
            from_email = extract_email_address(from_email_raw)
            
            # Clean the reply text (remove quoted content)
            cleaned_text = clean_reply_text(text_body)
            
            # Log the incoming email
            print(f"\n📧 Received email reply:")
            print(f"   From: {from_email}")
            print(f"   Subject: {subject}")
            print(f"   Cleaned text length: {len(cleaned_text)} chars")
            
            # Process asynchronously
            # Note: In production, you'd want to use a task queue (Celery, etc.)
            # For this example, we'll process synchronously but you can make it async
            import asyncio
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            result = loop.run_until_complete(
                process_email_reply(from_email, cleaned_text, subject)
            )
            loop.close()
            
            return jsonify({
                "status": "received",
                "processed": True,
                "result": result
            }), 200
            
        except Exception as e:
            print(f"❌ Error handling webhook: {str(e)}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500

    @app.route('/health', methods=['GET'])
    def health_check():
        """Health check endpoint"""
        return jsonify({"status": "healthy"}), 200

    if __name__ == '__main__':
        PORT = 5050  # Use 5050 to avoid conflicts on port 5000
        
        # Check if we're in a Jupyter notebook (disable reloader to avoid conflicts)
        import sys
        in_notebook = 'ipykernel' in sys.modules or 'IPython' in sys.modules
        
        print(f"🚀 Starting webhook server on http://localhost:{PORT}")
        print(f"📧 Webhook endpoint: http://localhost:{PORT}/webhook/email")
        print("💡 For local development, use ngrok to expose this server:")
        print(f"   ngrok http {PORT}")
        print("\n⚠️  Make sure to update your SendGrid Event Webhook URL to use this port!")
        print("\n💡 To stop the server, interrupt the kernel (Kernel → Interrupt)")
        
        # Disable reloader in notebooks to avoid ZMQ conflicts
        app.run(
            host='0.0.0.0', 
            port=PORT, 
            debug=True, 
            use_reloader=False  # Disable reloader in notebooks
        )


🚀 Starting webhook server on http://localhost:5050
📧 Webhook endpoint: http://localhost:5050/webhook/email
💡 For local development, use ngrok to expose this server:
   ngrok http 5050

⚠️  Make sure to update your SendGrid Event Webhook URL to use this port!

💡 To stop the server, interrupt the kernel (Kernel → Interrupt)
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5050
 * Running on http://192.168.1.21:5050
Press CTRL+C to quit


In [ ]:
# Update the send_email function to include reply-to for webhook routing
# This ensures replies go to your webhook endpoint

def send_email_with_reply_to(to_email: str, subject: str, body: str, reply_to_email: str = None):
    """
    Send an email with reply-to configured for webhook routing.
    
    Args:
        to_email: Recipient email
        subject: Email subject
        body: Email body
        reply_to_email: Email address that routes to webhook (optional)
    """
    try:
        api_key = os.environ.get('SENDGRID_API_KEY')
        if not api_key:
            return {"status": "error", "message": "SENDGRID_API_KEY not set"}
        
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        
        from_email = Email("vr.work.ams@gmail.com")  # Your verified sender
        to_email_obj = To(to_email)
        content = Content("text/plain", body)
        
        mail = Mail(from_email, to_email_obj, subject, content)
        
        # Set reply-to if provided (this should route to your inbound parse webhook)
        if reply_to_email:
            mail.reply_to = Email(reply_to_email)
        
        response = sg.client.mail.send.post(request_body=mail.get())
        
        return {
            "status": "success",
            "status_code": response.status_code
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

# Example: Send a sales email that will route replies to webhook
# Replace 'reply@yourdomain.com' with your SendGrid Inbound Parse hostname
# example_result = send_email_with_reply_to(
#     to_email="prospect@example.com",
#     subject="Quick question about SOC2 compliance",
#     body="Hi! I noticed you might be interested in SOC2 compliance tools...",
#     reply_to_email="reply@yourdomain.com"  # This routes to your webhook
# )


In [ ]:
# Test the webhook setup - Send a test email and track engagement

async def test_webhook_setup():
    """
    Send a test email and verify the webhook is ready to receive events.
    Make sure your webhook server (Cell 50) is running first!
    """
    print("🧪 Testing Webhook Setup")
    print("=" * 60)
    
    # Check if Flask app is running
    if Flask is None:
        print("❌ Flask is not installed. Run Cell 45 first!")
        return
    
    # Check if webhook server is accessible
    try:
        import requests
        response = requests.get("http://localhost:5000/health", timeout=2)
        if response.status_code == 200:
            print("✓ Webhook server is running!")
        else:
            print("⚠️  Webhook server responded but with unexpected status")
    except:
        print("⚠️  Webhook server not running. Start it by running Cell 50!")
        print("   Then run this test again.")
        return
    
    # Send a test email
    print("\n📧 Sending test email...")
    api_key = os.environ.get('SENDGRID_API_KEY')
    if not api_key:
        print("❌ SENDGRID_API_KEY not set")
        return
    
    try:
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        from_email = Email("vr.work.ams@gmail.com")  # Your verified sender
        to_email = To("vr.work.ams@gmail.com")  # Send to yourself for testing
        content = Content("text/plain", 
            "This is a test email to verify webhook setup.\n\n"
            "When you open this email, SendGrid will send an 'opened' event to your webhook, "
            "and you'll see a follow-up suggestion generated by the SDR agent!")
        mail = Mail(from_email, to_email, "Test: Webhook Setup", content).get()
        response = sg.client.mail.send.post(request_body=mail)
        
        if response.status_code == 202:
            print("✓ Test email sent successfully!")
            print("\n📋 Next steps:")
            print("   1. Check your email inbox")
            print("   2. OPEN the email (this triggers the webhook)")
            print("   3. Watch your Flask server terminal for the event and follow-up suggestion!")
            print("\n💡 If you don't see events:")
            print("   - Make sure ngrok is running and SendGrid is configured with the ngrok URL")
            print("   - Check SendGrid Dashboard → Activity to see if events are being sent")
        else:
            print(f"⚠️  Email send returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ Error sending test email: {str(e)}")

# Uncomment the line below to run the test
# await test_webhook_setup()
print("Test function ready! Uncomment the last line to run the test.")


### Testing the Webhook Locally

To test this locally:

1. **Start the webhook server:**
   ```python
   # Run the Flask app cell above
   # This starts the server on http://localhost:5000
   ```

2. **Expose your local server with ngrok:**
   ```bash
   ngrok http 5000
   ```
   This gives you a public URL like: `https://abc123.ngrok.io`

3. **Configure SendGrid Inbound Parse:**
   - Go to SendGrid Dashboard > Settings > Inbound Parse
   - Add a new hostname (or use your domain)
   - Set POST URL to: `https://abc123.ngrok.io/webhook/email`
   - Save

4. **Send a test email:**
   - Use `send_email_with_reply_to()` with a reply-to address that matches your inbound parse hostname
   - Reply to that email from another account
   - Watch the webhook receive and process the reply!

### Production Deployment

For production, consider:
- Deploying to a cloud service (AWS Lambda, Google Cloud Functions, Heroku, etc.)
- Using a proper task queue (Celery, RQ) for async processing
- Adding authentication/verification for webhook requests
- Storing conversation history in a database
- Adding rate limiting and error handling
- Using environment variables for configuration

### Next Steps

- Add conversation history tracking
- Implement multi-turn conversations
- Add sentiment analysis
- Create different response strategies based on reply type
- Integrate with CRM systems


## 📧 Setting Up SendGrid Event Webhook (No Domain Needed!)

### Step-by-Step Guide for Gmail Users:

#### Step 1: Install ngrok

**On macOS:**
```bash
brew install ngrok
```

**Or download from:** https://ngrok.com/download

**Or using pip:**
```bash
pip install pyngrok
```

#### Step 2: Start Your Local Webhook Server

Run the Flask app cell below (Cell 50) to start your webhook server on `http://localhost:5000`

#### Step 3: Expose Your Local Server with ngrok

Open a **new terminal** and run:
```bash
ngrok http 5000
```

You'll see output like:
```
Forwarding  https://abc123xyz.ngrok-free.app -> http://localhost:5000
```

**Copy the HTTPS URL** (e.g., `https://abc123xyz.ngrok-free.app`)

⚠️ **Important:** Keep ngrok running in that terminal window!

#### Step 4: Configure SendGrid Event Webhook

1. Go to **SendGrid Dashboard** → **Settings** → **Mail Settings** → **Event Webhook**
2. Click **"Create Webhook"** or **"Edit"** if one exists
3. Set **HTTP POST URL** to: `https://your-ngrok-url.ngrok-free.app/webhook/email`
   - Example: `https://abc123xyz.ngrok-free.app/webhook/email`
4. **Enable the events you want to track:**
   - ✅ **Delivered** - Email was successfully delivered
   - ✅ **Opened** - Recipient opened the email
   - ✅ **Clicked** - Recipient clicked a link
   - ✅ **Bounced** - Email bounced
   - ✅ **Dropped** - Email was dropped
   - ⚠️ **Note:** For email replies, you'll need Inbound Parse (requires domain)

5. Click **"Save"**

#### Step 5: Test Your Webhook

1. Send a test email using your existing code
2. Check your Flask server terminal - you should see webhook events coming in!
3. Check SendGrid Dashboard → **Activity** to see events

### 🎯 What You Can Track with Event Webhooks:

- **Email Delivery Status** - Know when emails are delivered
- **Open Rates** - See who opened your emails
- **Click Tracking** - Track which links were clicked
- **Bounce Handling** - Handle bounced emails automatically
- **Engagement Metrics** - Build analytics

### ⚠️ Limitation: Email Replies

**Event Webhooks cannot receive actual email reply content.** They only send metadata.

**To receive actual email replies (for automated responses), you need:**
1. A domain (can't use Gmail directly)
2. SendGrid Inbound Parse setup
3. DNS configuration

**Alternative for testing replies:**
- Use a service like **Mailgun** or **Postmark** that offer easier webhook setup
- Or get a free domain from services like **Freenom** or **Namecheap** (often $1-2/year)


In [ ]:
# Enhanced webhook handler for SendGrid Event Webhooks
# This handles events like delivered, opened, clicked, bounced, etc.

def handle_sendgrid_event(event_data):
    """
    Process SendGrid event webhook data.
    
    Event types include:
    - processed: Email was received and processed
    - delivered: Email was successfully delivered
    - opened: Recipient opened the email
    - clicked: Recipient clicked a link
    - bounce: Email bounced
    - dropped: Email was dropped
    - deferred: Delivery was deferred
    - unsubscribe: Recipient unsubscribed
    - spamreport: Recipient marked as spam
    """
    event_type = event_data.get('event')
    email = event_data.get('email', 'Unknown')
    timestamp = event_data.get('timestamp', 'Unknown')
    
    print(f"\n📬 SendGrid Event Received:")
    print(f"   Type: {event_type}")
    print(f"   Email: {email}")
    print(f"   Time: {timestamp}")
    
    # Handle different event types
    if event_type == 'delivered':
        print("   ✓ Email successfully delivered!")
    elif event_type == 'opened':
        print("   👁️  Email was opened!")
        # You could track this in a database
    elif event_type == 'clicked':
        url = event_data.get('url', 'Unknown')
        print(f"   🔗 Link clicked: {url}")
    elif event_type == 'bounce':
        reason = event_data.get('reason', 'Unknown')
        print(f"   ⚠️  Email bounced: {reason}")
    elif event_type == 'dropped':
        reason = event_data.get('reason', 'Unknown')
        print(f"   ❌ Email dropped: {reason}")
    
    return {
        "status": "processed",
        "event_type": event_type,
        "email": email
    }

# Example: Test event data structure
example_event = {
    "event": "opened",
    "email": "prospect@example.com",
    "timestamp": 1234567890,
    "sg_event_id": "abc123",
    "sg_message_id": "xyz789"
}

print("Example event structure:")
print(example_event)
